In [ ]:
!pip install google-cloud-vision
!pip install pillow
!pip install thefuzz

In [ ]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import types
import os
import io
import sys
import re
from PIL import Image, ImageDraw
import argparse
from enum import Enum
import json
from collections import defaultdict

In [ ]:
apikey = 'here'
def txtgrab(apikey,filename):
  client = vision_v1.ImageAnnotatorClient(client_options={"api_key": apikey})
  bounds = []
  with io.open(filename, 'rb') as image_file:
          content = image_file.read()
  image = types.Image(content=content)
  response = client.document_text_detection(image=image)
  document = response.full_text_annotation
  return [document,response,bounds]
[document,response,bounds] = txtgrab(apikey,'img_path')

#Getting coordinates for the words

In [ ]:
class FeatureType(Enum):
      PAGE = 1
      BLOCK = 2
      PARA = 3
      WORD = 4
      SYMBOL = 5

def draw_boxes(image, bounds, color):
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon(
            [
                bound.vertices[0].x,
                bound.vertices[0].y,
                bound.vertices[1].x,
                bound.vertices[1].y,
                bound.vertices[2].x,
                bound.vertices[2].y,
                bound.vertices[3].x,
                bound.vertices[3].y,
            ],
            None,
            color,
        )
    return image


def get_document_bounds(response, feature):
    jpg_image = Image.open("img_path")
    content = jpg_image.convert("RGB")
    dictionaryForCoordinates = defaultdict(list)
    for i,page in enumerate(document.pages):
        for block in page.blocks:
            if feature==FeatureType.BLOCK:
                bounds.append(block.bounding_box)
            for paragraph in block.paragraphs:
                if feature==FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)
                for word in paragraph.words:

                    ourword=""
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)
                        ourword=ourword+(symbol.text)
                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)
                        dictionaryForCoordinates[ourword].append(word.bounding_box)

    return [content,bounds,dictionaryForCoordinates]


content,bounds,dictionaryForCoordinates=get_document_bounds(response, FeatureType.WORD)
#draw_boxes(content,bounds,'yellow')

#Distance calculation for any given two words

In [ ]:
def distanceBetweenTwoWords(w1,w2):
  CoordinatesOfW1=dictionaryForCoordinates[w1]
  CoordinatesOfW2=dictionaryForCoordinates[w2]
  CoordinatesOfW1 = str(CoordinatesOfW1)
  CoordinatesOfW2 = str(CoordinatesOfW2)
  W1xvalues = [int(x) for x in re.findall(r'x: (\d+)', CoordinatesOfW1)]
  W1yvalues = [int(y) for y in re.findall(r'y: (\d+)', CoordinatesOfW1)]
  W2xvalues = [int(x) for x in re.findall(r'x: (\d+)', CoordinatesOfW2)]
  W2yvalues = [int(y) for y in re.findall(r'y: (\d+)', CoordinatesOfW2)]
  horizontalCenterOfW1 = (W1xvalues[1]+W1xvalues[0])/2
  horizontalCenterOfW2 = (W2xvalues[1]+W2xvalues[0])/2
  verticalCenterOfW1 = (W1yvalues[2]+W1yvalues[1])/2
  verticalCenterOfW2 = (W2yvalues[2]+W2yvalues[1])/2
  horizontalDistance = abs(horizontalCenterOfW2-horizontalCenterOfW1)
  verticalDistance = abs(verticalCenterOfW2-verticalCenterOfW1)

  return [horizontalDistance,verticalDistance]

def distanceBetweenTwoWordsAllIterations(w1,w2):
  AllCoordinatesOfW1=dictionaryForCoordinates[w1]
  AllCoordinatesOfW2=dictionaryForCoordinates[w2]
  AllDistances = []
  for i in AllCoordinatesOfW1:
    for j in AllCoordinatesOfW2:
        CoordinatesOfW1 = str(i)
        CoordinatesOfW2 = str(j)

        W1xvalues = [int(x) for x in re.findall(r'x: (\d+)', CoordinatesOfW1)]
        W1yvalues = [int(y) for y in re.findall(r'y: (\d+)', CoordinatesOfW1)]
        W2xvalues = [int(x) for x in re.findall(r'x: (\d+)', CoordinatesOfW2)]
        W2yvalues = [int(y) for y in re.findall(r'y: (\d+)', CoordinatesOfW2)]

        horizontalCenterOfW1 = (W1xvalues[1]+W1xvalues[0])/2
        horizontalCenterOfW2 = (W2xvalues[1]+W2xvalues[0])/2
        verticalCenterOfW1 = (W1yvalues[2]+W1yvalues[1])/2
        verticalCenterOfW2 = (W2yvalues[2]+W2yvalues[1])/2
        horizontalDistance = abs(horizontalCenterOfW2-horizontalCenterOfW1)
        verticalDistance = abs(verticalCenterOfW2-verticalCenterOfW1)
        AllDistances.append([horizontalDistance,verticalDistance])
  return AllDistances


print(distanceBetweenTwoWordsAllIterations('Word1','Word2'))



def sameLine(w1,w2):
  for i in distanceBetweenTwoWordsAllIterations(w1,w2):
    if i[0]>10 and i[1]<15:
      return [True,i]
  else:
    return [False,i]

# User X